In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore")
df=pd.read_csv("model\dataset_cleaned.csv")

# Features (drop targets + identifiers that don't help)
X = df.drop(columns=["Leakage_Flag", "Anomaly_Type", "Invoice_Number", "Billing_Date", "Billing_Time"])

# Targets (multi-output)
y = df[["Leakage_Flag", "Anomaly_Type"]]

# Identify categorical and numeric columns
cat_cols = X.select_dtypes(include=["object"]).columns
num_cols = X.select_dtypes(exclude=["object"]).columns

# Preprocessor (OneHot for categorical, Scale for numeric)
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", StandardScaler(), num_cols)
    ]
)

# Build pipeline with RandomForest multi-output classifier
pipeline = Pipeline([
    ("preprocess", preprocessor),
    ("clf", MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42)))
])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y["Leakage_Flag"])

# Fit model
pipeline.fit(X_train, y_train)

# Predictions
y_pred = pipeline.predict(X_test)

# Evaluation
print("Leakage_Flag Report:")
print(classification_report(y_test["Leakage_Flag"], y_pred[:, 0]))

print("Anomaly_Type Report:")
print(classification_report(y_test["Anomaly_Type"], y_pred[:, 1]))


Leakage_Flag Report:
              precision    recall  f1-score   support

     Anomaly       0.96      0.60      0.74       259
  No Leakage       0.81      0.99      0.89       437

    accuracy                           0.84       696
   macro avg       0.88      0.79      0.81       696
weighted avg       0.86      0.84      0.83       696

Anomaly_Type Report:
                         precision    recall  f1-score   support

      Duplicate Entries       0.91      0.94      0.93        33
         Excess Payment       1.00      0.94      0.97        47
        Missing Charges       1.00      0.03      0.06        31
             No Anomaly       0.78      1.00      0.88       437
Payment Status Mismatch       1.00      0.93      0.96        41
          Under Payment       1.00      0.35      0.51        55
   Unrecorded Discounts       0.00      0.00      0.00        52

               accuracy                           0.82       696
              macro avg       0.81      0.60

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings("ignore")
df=pd.read_csv("model\dataset_cleaned.csv")

# Features (drop targets + identifiers that don't help)
X = df.drop(columns=["Leakage_Flag", "Anomaly_Type", "Invoice_Number", "Billing_Date", "Billing_Time"])

# Encode target columns
leakage_encoder = LabelEncoder()
anomaly_encoder = LabelEncoder()

y = pd.DataFrame({
    "Leakage_Flag": leakage_encoder.fit_transform(df["Leakage_Flag"]),
    "Anomaly_Type": anomaly_encoder.fit_transform(df["Anomaly_Type"])
})

# Identify categorical and numeric columns
cat_cols = X.select_dtypes(include=["object"]).columns
num_cols = X.select_dtypes(exclude=["object"]).columns

# Preprocessor (OneHot for categorical, Scale for numeric)
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", StandardScaler(), num_cols)
    ]
)

# XGBoost Classifier
xgb = XGBClassifier(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    use_label_encoder=False,
    eval_metric="mlogloss"
)

# Multi-output wrapper
multi_xgb = MultiOutputClassifier(xgb)

# Build pipeline
pipeline = Pipeline([
    ("preprocess", preprocessor),
    ("clf", multi_xgb)
])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y["Leakage_Flag"]
)

# Fit model
pipeline.fit(X_train, y_train)

# Predictions (numeric)
y_pred = pipeline.predict(X_test)

# Convert back to original labels
y_pred_df = pd.DataFrame({
    "Leakage_Flag": leakage_encoder.inverse_transform(y_pred[:, 0]),
    "Anomaly_Type": anomaly_encoder.inverse_transform(y_pred[:, 1])
})

y_test_decoded = pd.DataFrame({
    "Leakage_Flag": leakage_encoder.inverse_transform(y_test["Leakage_Flag"]),
    "Anomaly_Type": anomaly_encoder.inverse_transform(y_test["Anomaly_Type"])
})

# Evaluation
print("Leakage_Flag Report:")
print(classification_report(y_test_decoded["Leakage_Flag"], y_pred_df["Leakage_Flag"]))

print("Anomaly_Type Report:")
print(classification_report(y_test_decoded["Anomaly_Type"], y_pred_df["Anomaly_Type"]))


Leakage_Flag Report:
              precision    recall  f1-score   support

     Anomaly       0.99      0.79      0.88       259
  No Leakage       0.89      1.00      0.94       437

    accuracy                           0.92       696
   macro avg       0.94      0.89      0.91       696
weighted avg       0.93      0.92      0.92       696

Anomaly_Type Report:
                         precision    recall  f1-score   support

      Duplicate Entries       1.00      1.00      1.00        33
         Excess Payment       1.00      1.00      1.00        47
        Missing Charges       0.88      0.48      0.62        31
             No Anomaly       0.88      1.00      0.93       437
Payment Status Mismatch       1.00      1.00      1.00        41
          Under Payment       1.00      1.00      1.00        55
   Unrecorded Discounts       0.86      0.12      0.20        52

               accuracy                           0.91       696
              macro avg       0.95      0.80

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore")
df=pd.read_csv("model\dataset_cleaned.csv")

# Handle missing values
df['Service_Category'].fillna('No Service', inplace=True)
df['Anomaly_Type'].fillna('No Anomaly', inplace=True)

# Identify categorical columns for one-hot encoding
categorical_cols = [
    'Payment_Status', 'Transaction_Type', 'Mode_of_Payment', 'Product_Name',
    'Service', 'Product_Category', 'Service_Category', 'Store_Branch',
    'Cashier_ID', 'Supplier_ID', 'Customer_Type', 'Order_Channel', 'Anomaly_Type'
]
df_preprocessed = pd.get_dummies(df, columns=categorical_cols, dtype=int)

# Drop non-predictive identifier and time-based columns
columns_to_drop = [
    'Invoice_Number', 'Customer_ID', 'Service_ID', 'Billing_Date', 'Billing_Time'
]
df_preprocessed = df_preprocessed.drop(columns=columns_to_drop, errors='ignore')

# Convert the target variable 'Leakage_Flag' to a binary numerical format
df_preprocessed['Leakage_Flag'] = df_preprocessed['Leakage_Flag'].replace({'No Leakage': 0, 'Anomaly': 1})

# --- Step 2: Split Data ---
# Separate features (X) and target (y)
X = df_preprocessed.drop('Leakage_Flag', axis=1)
y = df_preprocessed['Leakage_Flag']

# Split the data into a training set and a testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Step 3: Train the Model ---
# Initialize and train the Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# --- Step 4: Make Predictions and Generate Report ---
# Use the trained model to make predictions on the test set
y_pred = model.predict(X_test)

# Generate and print the classification report which includes Recall and F1-Score
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       437
           1       1.00      1.00      1.00       259

    accuracy                           1.00       696
   macro avg       1.00      1.00      1.00       696
weighted avg       1.00      1.00      1.00       696



In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore")

# --- Step 1: Load & Preprocess ---
df = pd.read_csv("model/dataset_cleaned.csv")

# Handle missing values
df['Service_Category'].fillna('No Service', inplace=True)
df['Anomaly_Type'].fillna('No Anomaly', inplace=True)

# Convert the target variable 'Leakage_Flag' to binary (0 = No Leakage, 1 = Anomaly)
df['Leakage_Flag'] = df['Leakage_Flag'].replace({'No Leakage': 0, 'Anomaly': 1})

# Identify categorical columns for one-hot encoding
categorical_cols = [
    'Payment_Status', 'Transaction_Type', 'Mode_of_Payment', 'Product_Name',
    'Service', 'Product_Category', 'Service_Category', 'Store_Branch',
    'Cashier_ID', 'Supplier_ID', 'Customer_Type', 'Order_Channel'
    # ⚠️ Removed 'Anomaly_Type' to avoid data leakage
]

# One-hot encode categorical variables
df_preprocessed = pd.get_dummies(df, columns=categorical_cols, dtype=int)

# Drop identifiers and time-based columns (not predictive)
columns_to_drop = ['Invoice_Number', 'Customer_ID', 'Service_ID', 'Billing_Date', 'Billing_Time']
df_preprocessed = df_preprocessed.drop(columns=columns_to_drop, errors='ignore')

# --- Step 2: Define Features and Target ---
# Exclude both target columns from features
X = df_preprocessed.drop(columns=['Leakage_Flag', 'Anomaly_Type'])
y = df_preprocessed['Leakage_Flag']

# --- Step 3: Train-Test Split ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# --- Step 4: Train the Model ---
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# --- Step 5: Predictions & Evaluation ---
y_pred = model.predict(X_test)

print("Classification Report for Leakage_Flag (without Anomaly_Type as feature):")
print(classification_report(y_test, y_pred))


Classification Report for Leakage_Flag (without Anomaly_Type as feature):
              precision    recall  f1-score   support

           0       0.83      1.00      0.91       437
           1       0.99      0.66      0.80       259

    accuracy                           0.87       696
   macro avg       0.91      0.83      0.85       696
weighted avg       0.89      0.87      0.87       696



In [14]:
df.columns

Index(['Invoice_Number', 'Customer_ID', 'Service_ID', 'Billing_Date',
       'Payment_Status', 'Leakage_Flag', 'Transaction_Type', 'Mode_of_Payment',
       'Product_Name', 'Service', 'Product_Category', 'Service_Category',
       'Product_Quantity', 'Tax_Amount', 'Actual_Amount', 'Billed_Amount',
       'Paid_Amount', 'Balance_Amount', 'Unit_Price', 'Tax_Rate',
       'Service_Charge', 'Discount_Amount', 'Store_Branch', 'Cashier_ID',
       'Supplier_ID', 'Billing_Time', 'Customer_Type', 'Order_Channel',
       'Anomaly_Type', 'Invoice_Num_Int', 'Is_Duplicate'],
      dtype='object')

In [27]:

df['actual_billing_amnt'] = df['Actual_Amount'] + df['Tax_Amount'] + df['Service_Charge'] - df['Discount_Amount']


In [16]:
# Loop through each unique value in Service column and display rows
unique_services = df['Anomaly_Type'].unique()
pd.set_option('display.max_columns', None)

for service in unique_services:
    print(f"\n===== Rows for Service: {service} =====\n")
    display(df[df['Anomaly_Type'] == service].head())  # show first few rows for each service


===== Rows for Service: No Anomaly =====



,Invoice_Number,Customer_ID,Service_ID,Billing_Date,Payment_Status,Leakage_Flag,Transaction_Type,Mode_of_Payment,Product_Name,Service,Product_Category,Service_Category,Product_Quantity,Tax_Amount,Actual_Amount,Billed_Amount,Paid_Amount,Balance_Amount,Unit_Price,Tax_Rate,Service_Charge,Discount_Amount,Store_Branch,Cashier_ID,Supplier_ID,Billing_Time,Customer_Type,Order_Channel,Anomaly_Type,Invoice_Num_Int,Is_Duplicate,actual_billing_amnt
0,INV10002,CUST8861,SRV002,2023-12-17,Paid,0,Sale,Credit Card,Tea Powder (250g),Standard Delivery,Beverages,Delivery Services,4,24.0,480.0,544.0,544.0,0.0,120.0,0.05,40,0,Branch B,CASH011,SUP143,16:09:25,Premium,In-Store,No Anomaly,10002,0,544.0
1,INV10008,CUST6820,SRV003,2024-05-23,Failed,0,Sale,Net Banking,Chips (100g),Express Delivery,Snacks,Delivery Services,1,3.6,20.0,103.6,0.0,103.6,20.0,0.18,80,0,Branch B,CASH009,SUP166,19:20:46,Regular,Phone,No Anomaly,10008,0,103.6
3,INV10014,CUST8666,SRV003,2023-11-08,Failed,0,Sale,UPI,Dishwash Soap (500ml),Express Delivery,Household,Delivery Services,3,43.2,240.0,363.2,0.0,363.2,80.0,0.18,80,0,Online,CASH003,SUP156,14:10:44,Wholesale,In-Store,No Anomaly,10014,0,363.2
4,INV10019,CUST7397,SRV002,2024-02-12,Paid,0,Return,Debit Card,Lentils (1kg),Standard Delivery,Grains & Cereals,Delivery Services,9,162.0,900.0,1102.0,1102.0,0.0,100.0,0.18,40,0,Online,CASH013,SUP160,19:03:48,Wholesale,Phone,No Anomaly,10019,0,1102.0
9,INV10033,CUST6461,SRV001,2024-09-13,Paid,0,Sale,Net Banking,Apples (1kg),No Service,Fruits,No Service,9,194.4,1080.0,1274.4,1274.4,0.0,120.0,0.18,0,0,Branch B,CASH019,SUP161,10:19:07,New,In-Store,No Anomaly,10033,0,1274.4



===== Rows for Service: Excess Payment =====



,Invoice_Number,Customer_ID,Service_ID,Billing_Date,Payment_Status,Leakage_Flag,Transaction_Type,Mode_of_Payment,Product_Name,Service,Product_Category,Service_Category,Product_Quantity,Tax_Amount,Actual_Amount,Billed_Amount,Paid_Amount,Balance_Amount,Unit_Price,Tax_Rate,Service_Charge,Discount_Amount,Store_Branch,Cashier_ID,Supplier_ID,Billing_Time,Customer_Type,Order_Channel,Anomaly_Type,Invoice_Num_Int,Is_Duplicate,actual_billing_amnt
2,INV10009,CUST4895,SRV001,2023-01-13,Paid,1,Return,UPI,Bananas (1kg),No Service,Fruits,No Service,4,10.0,200.0,210.0,248.658291,-38.658291,50.0,0.05,0,0,Online,CASH011,SUP109,18:52:28,Regular,Online,Excess Payment,10009,0,210.0
11,INV10046,CUST3151,SRV002,2023-10-24,Paid,1,Sale,Cheque,Salt (1kg),Standard Delivery,Pantry Staples,Delivery Services,9,21.6,180.0,241.6,276.307189,-34.707189,20.0,0.12,40,0,Online,CASH016,SUP149,19:30:23,Premium,Online,Excess Payment,10046,0,241.6
14,INV10052,CUST6200,SRV001,2023-04-23,Paid,1,Return,Net Banking,Milk (1L),No Service,Dairy & Eggs,No Service,10,108.0,600.0,708.0,823.938950,-115.938950,60.0,0.18,0,0,Main Store,CASH007,SUP123,20:17:25,New,In-Store,Excess Payment,10052,0,708.0
23,INV10081,CUST1403,SRV001,2024-08-15,Paid,1,Sale,Cash,Cheese (200g),No Service,Dairy & Eggs,No Service,9,243.0,1350.0,1593.0,1834.950649,-241.950649,150.0,0.18,0,0,Main Store,CASH003,SUP135,08:32:45,Regular,In-Store,Excess Payment,10081,0,1593.0
27,INV10089,CUST3511,SRV003,2023-10-24,Paid,1,Sale,UPI,Biscuits (200g),Express Delivery,Snacks,Delivery Services,4,6.0,120.0,206.0,238.726332,-32.726332,30.0,0.05,80,0,Branch A,CASH018,SUP174,03:36:09,Regular,Online,Excess Payment,10089,0,206.0



===== Rows for Service: Missing Charges =====



...

,Invoice_Number,Customer_ID,Service_ID,Billing_Date,Payment_Status,Leakage_Flag,Transaction_Type,Mode_of_Payment,Product_Name,Service,Product_Category,Service_Category,Product_Quantity,Tax_Amount,Actual_Amount,Billed_Amount,Paid_Amount,Balance_Amount,Unit_Price,Tax_Rate,Service_Charge,Discount_Amount,Store_Branch,Cashier_ID,Supplier_ID,Billing_Time,Customer_Type,Order_Channel,Anomaly_Type,Invoice_Num_Int,Is_Duplicate,actual_billing_amnt
7,INV10031,CUST3570,SRV001,2023-11-24,Paid,1,Return,Credit Card,Fish (1kg),No Service,Meat & Fish,No Service,9,324.0,2700.0,3024.000000,3024.000000,0.0,300.0,0.12,0,0,Online,CASH004,SUP115,18:39:27,New,Online,Duplicate Entries,10031,1,3024.0
8,INV10031,CUST4268,SRV003,2024-02-10,Paid,1,Sale,Credit Card,Coffee Powder (200g),Express Delivery,Beverages,Delivery Services,1,17.5,350.0,369.685152,369.685152,0.0,350.0,0.05,80,0,Branch B,CASH002,SUP189,16:16:16,Premium,Online,Duplicate Entries,10031,1,447.5
29,INV10097,CUST2899,SRV002,2023-12-31,Paid,1,Sale,Net Banking,Eggs (12 pack),Standard Delivery,Dairy & Eggs,Delivery Services,4,33.6,280.0,353.600000,353.600000,0.0,70.0,0.12,40,0,Main Store,CASH010,SUP152,13:06:14,Premium,In-Store,Duplicate Entries,10097,1,353.6
30,INV10097,CUST8604,SRV003,2024-08-27,Pending,1,Sale,Credit Card,Apples (1kg),Express Delivery,Fruits,Delivery Services,6,86.4,720.0,886.400000,0.000000,886.4,120.0,0.12,80,0,Branch B,CASH012,SUP160,10:14:47,Regular,Phone,Duplicate Entries,10097,1,886.4
59,INV10169,CUST2363,SRV001,2023-06-05,Pending,1,Return,Cheque,Cheese (200g),No Service,Dairy & Eggs,No Service,1,27.0,150.0,177.000000,0.000000,177.0,150.0,0.18,0,0,Main Store,CASH012,SUP181,09:45:04,Premium,Online,Duplicate Entries,10169,1,177.0



===== Rows for Service: Unrecorded Discounts =====



,Invoice_Number,Customer_ID,Service_ID,Billing_Date,Payment_Status,Leakage_Flag,Transaction_Type,Mode_of_Payment,Product_Name,Service,Product_Category,Service_Category,Product_Quantity,Tax_Amount,Actual_Amount,Billed_Amount,Paid_Amount,Balance_Amount,Unit_Price,Tax_Rate,Service_Charge,Discount_Amount,Store_Branch,Cashier_ID,Supplier_ID,Billing_Time,Customer_Type,Order_Channel,Anomaly_Type,Invoice_Num_Int,Is_Duplicate,actual_billing_amnt
12,INV10048,CUST6461,SRV001,2024-07-04,Failed,1,Sale,Credit Card,Coffee Powder (200g),No Service,Beverages,No Service,5,315.0,1750.0,1835.645773,0.000000,1835.645773,350.0,0.18,0,0,Branch A,CASH013,SUP182,12:28:51,New,Online,Unrecorded Discounts,10048,0,2065.0
21,INV10071,CUST7457,SRV001,2024-02-20,Pending,1,Return,Cash,Apples (1kg),No Service,Fruits,No Service,8,172.8,960.0,991.219543,0.000000,991.219543,120.0,0.18,0,0,Branch B,CASH011,SUP103,04:48:54,Regular,Online,Unrecorded Discounts,10071,0,1132.8
32,INV10105,CUST3255,SRV003,2023-05-14,Failed,1,Sale,Cash,Biscuits (200g),Express Delivery,Snacks,Delivery Services,1,3.6,30.0,107.864700,0.000000,107.864700,30.0,0.12,80,0,Online,CASH001,SUP105,04:13:43,Wholesale,In-Store,Unrecorded Discounts,10105,0,113.6
40,INV10122,CUST9232,SRV003,2023-06-17,Paid,1,Sale,Cheque,Potatoes (1kg),Express Delivery,Fresh Vegetables,Delivery Services,1,6.3,35.0,115.670169,115.670169,0.000000,35.0,0.18,80,0,Branch B,CASH014,SUP116,11:51:40,Premium,Phone,Unrecorded Discounts,10122,0,121.3
62,INV10173,CUST3407,SRV001,2023-08-11,Failed,1,Sale,UPI,Toothpaste (150g),No Service,Personal Care,No Service,8,24.0,480.0,390.489796,0.000000,390.489796,60.0,0.05,0,0,Branch B,CASH013,SUP126,20:00:37,Premium,In-Store,Unrecorded Discounts,10173,0,504.0



===== Rows for Service: Payment Status Mismatch =====



,Invoice_Number,Customer_ID,Service_ID,Billing_Date,Payment_Status,Leakage_Flag,Transaction_Type,Mode_of_Payment,Product_Name,Service,Product_Category,Service_Category,Product_Quantity,Tax_Amount,Actual_Amount,Billed_Amount,Paid_Amount,Balance_Amount,Unit_Price,Tax_Rate,Service_Charge,Discount_Amount,Store_Branch,Cashier_ID,Supplier_ID,Billing_Time,Customer_Type,Order_Channel,Anomaly_Type,Invoice_Num_Int,Is_Duplicate,actual_billing_amnt
45,INV10136,CUST6855,SRV003,2023-05-24,Failed,1,Return,Credit Card,Milk (1L),Express Delivery,Dairy & Eggs,Delivery Services,3,9.0,180.0,269.0,133.668679,135.331321,60.0,0.05,80,0,Branch A,CASH006,SUP121,07:03:27,Wholesale,Online,Payment Status Mismatch,10136,0,269.0
49,INV10145,CUST7660,SRV001,2023-09-19,Failed,1,Return,Debit Card,Detergent (1kg),No Service,Household,No Service,3,72.0,600.0,672.0,193.380396,478.619604,200.0,0.12,0,0,Main Store,CASH007,SUP160,00:09:55,Wholesale,In-Store,Payment Status Mismatch,10145,0,672.0
51,INV10152,CUST8643,SRV003,2024-01-29,Failed,1,Return,Cheque,Cooking Oil (1L),Express Delivery,Pantry Staples,Delivery Services,4,30.0,600.0,710.0,251.945323,458.054677,150.0,0.05,80,0,Branch A,CASH008,SUP127,05:12:11,Regular,In-Store,Payment Status Mismatch,10152,0,710.0
61,INV10171,CUST2396,SRV003,2023-08-08,Failed,1,Return,Debit Card,Chicken (1kg),Express Delivery,Meat & Fish,Delivery Services,3,90.0,750.0,920.0,385.085309,534.914691,250.0,0.12,80,0,Branch B,CASH005,SUP194,05:00:53,Wholesale,Online,Payment Status Mismatch,10171,0,920.0
78,INV10217,CUST3109,SRV001,2023-01-19,Failed,1,Return,Debit Card,Tea Powder (250g),No Service,Beverages,No Service,6,86.4,720.0,806.4,128.891960,677.508040,120.0,0.12,0,0,Main Store,CASH006,SUP159,10:53:08,Premium,In-Store,Payment Status Mismatch,10217,0,806.4


In [23]:
# Loop through each unique value in Anomaly_Type column and display only Billed_Amount and actual_billing_amnt
unique_anomalies = df['Anomaly_Type'].unique()
pd.set_option('display.max_columns', None)

for anomaly in unique_anomalies:
    print(f"\n===== Rows for Anomaly_Type: {anomaly} =====\n")
    display(df[df['Anomaly_Type'] == anomaly][['Billed_Amount', 'actual_billing_amnt']].head())



===== Rows for Anomaly_Type: No Anomaly =====



KeyError: "['actual_billing_amnt'] not in index"

In [24]:
# Replace 'Unrecorded Discounts' with 'Missing Charges' in Anomaly_Type column
df['Anomaly_Type'] = df['Anomaly_Type'].replace('Unrecorded Discounts', 'Missing Charges')

# Optional: verify the change
print(df['Anomaly_Type'].unique())


['No Anomaly' 'Excess Payment' 'Missing Charges' 'Under Payment'
 'Duplicate Entries' 'Payment Status Mismatch']


In [21]:
# Filter dataset for 'Payment Status Mismatch'
payment_mismatch_df = df[df['Anomaly_Type'] == 'Payment Status Mismatch']

# Optional: check first few rows
payment_mismatch_df['Payment_Status'].unique()

array(['Failed'], dtype=object)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings("ignore")
#df=pd.read_csv("model\dataset_cleaned.csv")

# Features (drop targets + identifiers that don't help)
X = df.drop(columns=["Leakage_Flag", "Anomaly_Type", "Invoice_Number", "Billing_Date", "Billing_Time"])

# Encode target columns
leakage_encoder = LabelEncoder()
anomaly_encoder = LabelEncoder()

y = pd.DataFrame({
    "Leakage_Flag": leakage_encoder.fit_transform(df["Leakage_Flag"]),
    "Anomaly_Type": anomaly_encoder.fit_transform(df["Anomaly_Type"])
})

# Identify categorical and numeric columns
cat_cols = X.select_dtypes(include=["object"]).columns
num_cols = X.select_dtypes(exclude=["object"]).columns

# Preprocessor (OneHot for categorical, Scale for numeric)
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", StandardScaler(), num_cols)
    ]
)

# XGBoost Classifier
xgb = XGBClassifier(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    use_label_encoder=False,
    eval_metric="mlogloss"
)

# Multi-output wrapper
multi_xgb = MultiOutputClassifier(xgb)

# Build pipeline
pipeline = Pipeline([
    ("preprocess", preprocessor),
    ("clf", multi_xgb)
])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y["Leakage_Flag"]
)

# Fit model
pipeline.fit(X_train, y_train)

# Predictions (numeric)
y_pred = pipeline.predict(X_test)

# Convert back to original labels
y_pred_df = pd.DataFrame({
    "Leakage_Flag": leakage_encoder.inverse_transform(y_pred[:, 0]),
    "Anomaly_Type": anomaly_encoder.inverse_transform(y_pred[:, 1])
})

y_test_decoded = pd.DataFrame({
    "Leakage_Flag": leakage_encoder.inverse_transform(y_test["Leakage_Flag"]),
    "Anomaly_Type": anomaly_encoder.inverse_transform(y_test["Anomaly_Type"])
})

# Evaluation
print("Leakage_Flag Report:")
print(classification_report(y_test_decoded["Leakage_Flag"], y_pred_df["Leakage_Flag"]))

print("Anomaly_Type Report:")
print(classification_report(y_test_decoded["Anomaly_Type"], y_pred_df["Anomaly_Type"]))


Leakage_Flag Report:
              precision    recall  f1-score   support

     Anomaly       0.98      0.85      0.91       259
  No Leakage       0.92      0.99      0.95       437

    accuracy                           0.94       696
   macro avg       0.95      0.92      0.93       696
weighted avg       0.94      0.94      0.94       696

Anomaly_Type Report:
                         precision    recall  f1-score   support

      Duplicate Entries       1.00      1.00      1.00        33
         Excess Payment       1.00      1.00      1.00        47
        Missing Charges       0.98      0.58      0.73        83
             No Anomaly       0.93      1.00      0.96       437
Payment Status Mismatch       1.00      1.00      1.00        41
          Under Payment       1.00      1.00      1.00        55

               accuracy                           0.95       696
              macro avg       0.98      0.93      0.95       696
           weighted avg       0.95      0.95

In [28]:
df.columns

Index(['Invoice_Number', 'Customer_ID', 'Service_ID', 'Billing_Date',
       'Payment_Status', 'Leakage_Flag', 'Transaction_Type', 'Mode_of_Payment',
       'Product_Name', 'Service', 'Product_Category', 'Service_Category',
       'Product_Quantity', 'Tax_Amount', 'Actual_Amount', 'Billed_Amount',
       'Paid_Amount', 'Balance_Amount', 'Unit_Price', 'Tax_Rate',
       'Service_Charge', 'Discount_Amount', 'Store_Branch', 'Cashier_ID',
       'Supplier_ID', 'Billing_Time', 'Customer_Type', 'Order_Channel',
       'Anomaly_Type', 'Invoice_Num_Int', 'Is_Duplicate',
       'actual_billing_amnt'],
      dtype='object')

In [31]:
df.to_csv("supermarket_dataset.csv", index=False)